In [1]:
import dask.dataframe as dd
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
import holidays
from pathlib import Path

C:\Users\jmvdp\anaconda3\Lib\site-packages\holidays\__init__.py:32: FutureIncompatibilityWarning: 

This is a future version incompatibility warning from Python Holidays library v0.52
to inform you about an upcoming change in our API versioning strategy that may affect your
project's dependencies. Starting from version 1.0 onwards, we will be following a loose form of
Semantic Versioning (SemVer, https://semver.org) to provide clearer communication regarding any
potential breaking changes.

This means that while we strive to maintain backward compatibility, there might be occasional
updates that introduce breaking changes to our API. To ensure the stability of your projects,
we highly recommend pinning the version of our API that you rely on. You can pin your current
holidays v0.x dependency (e.g., holidays==0.52) or limit it (e.g., holidays<1.0) in order to
avoid potentially unwanted upgrade to the version 1.0 when it's released (ETA 2024Q4 - 2025Q1).

If you have any questions or con

In [ ]:
# Uncomment if  "Modelação_GOPI_FP_Granularidade_Subseccao_GROWTH" has not run first
# Import original dataset
# manipulated_data_directory = base_dir / "NaMinhaRua" / "Ficheiros de dados Manipulados"
# gopidataset = pd.read_csv(manipulated_data_directory/'combined_sorted_ym.gopi_data.csv')
# gopidataset.head(2)

In [ ]:
# Uncomment if  "Modelação_GOPI_FP_Granularidade_Subseccao_GROWTH" has not run first
# Create new data model
# gopidatasetmodel=gopidataset[['dt_registo', 'Subseccao', 'Freguesia', 'tipo']]
# gopidatasetmodel.head(2)
# gopidatasetmodel.to_csv('gopidatasetmodel1.csv', index=False)

In [4]:
#################################################Handling the new data model#######################################
base_dir = Path.home()
manipulated_data_directory = base_dir / "NaMinhaRua" / "Ficheiros de dados Manipulados"
gopidatasetbasemodel1 = pd.read_csv(manipulated_data_directory/ 'gopidatasetmodel1.csv')
gopidatasetbasemodel1.head(2)

,dt_registo,Subseccao,Freguesia,tipo
0,2018-01-01,11066202005,Parque das Nações,Candeeiro apagado
1,2018-01-01,11061802202,Lumiar,"Árvores, arbustos ou relva - Manutenção"


In [ ]:

# Group by 'tipo' and count occurrences
tipo_frequencia = gopidatasetbasemodel1['tipo'].value_counts()

# Calculate total frequency
total_ocorrencias = tipo_frequencia.sum()

# Calculate the percentage for each type
tipo_percentagem = (tipo_frequencia / total_ocorrencias) * 100

# Show the 20 types with the highest percentage
top_20_percentagem = tipo_percentagem.head(20)

print(top_20_percentagem)

In [ ]:
# Formate dt_registo as date
gopidatasetbasemodel1['dt_registo'] = pd.to_datetime(gopidatasetbasemodel1['dt_registo'], format='%Y-%m-%d')

# Create the "tipo_frequencia_diaria" with occurrence count
tipo_frequencia_diaria = gopidatasetbasemodel1.groupby(['dt_registo', 'Freguesia', 'tipo']).size().reset_index(name='counts')

# Select the 20 most frequent types
top_20_tipos = tipo_frequencia_diaria.groupby('tipo')['counts'].sum().nlargest(20).index

# Filter data to include only the top 20 'tipos'
tipo_frequencia_diaria = tipo_frequencia_diaria[tipo_frequencia_diaria['tipo'].isin(top_20_tipos)]

# Create a pivot 
pivot_df = tipo_frequencia_diaria.pivot_table(index=['dt_registo', 'Freguesia'], columns='tipo', values='counts', fill_value=0)

# Check column names before changing
print("Antes da alteração:", pivot_df.columns)

# Renaming columns to initials
def rename_to_initials(col):
   # Substituir caracteres especiais por underscores
    col = col.replace(' ', '_').replace('-', '_').replace(',', '_').replace('(', '_').replace(')', '_').replace('ó', 'o').replace('õ', 'o').replace('ã', 'a').replace('á', 'a').replace('ç', 'c').replace('í', 'i').replace('â', 'a').replace('é', 'e').replace('ú', 'u')
    # Split the column name into words.
    words = col.split('_')
    # Get the first letter of each word and combine all the initials
    initials = ''.join([word[0].upper() for word in words if word])
    return initials

pivot_df.columns = [rename_to_initials(col) for col in pivot_df.columns]

# Check column names after change
print("Depois da alteração:", pivot_df.columns)

# Merge with the original DataFrame filling unmatched cells with zeros
gopidataset_group = pd.merge(gopidatasetbasemodel1, pivot_df, on=['dt_registo', 'Freguesia'], how='left').fillna(0)

# Check the results
print(gopidataset_group.head(3))

# Remove the column 'tipo'
gopidataset_group.drop(columns=['tipo'], inplace=True)

In [ ]:
# Calculate the total sum of occurrences for each 'tipo' in the top 20
total_counts_top_20 = tipo_frequencia_diaria.groupby('tipo')['counts'].sum()

# Calculate the grand total of occurences for the 20 'tipo'
total_counts = total_counts_top_20.sum()

# Calculate the percentagem for each 'tipo' within the top 20
percentages_top_20 = (total_counts_top_20 / total_counts) * 100

# Sort the percentages in descending order
percentages_top_20_sorted = percentages_top_20.sort_values(ascending=False)

# Display percentages for each 'tipo' in the top 20
print(percentages_top_20_sorted )

In [ ]:
# Function to check if the day is a weekend
def is_weekend(date):
    return date.weekday() >= 5  # 5 = Sábado, 6 = Domingo

gopidataset_group['Fim_de_Semana'] = gopidataset_group['dt_registo'].apply(is_weekend)

# Count how many times Fim_de_Semana is True and False
num_ocorrencias = gopidataset_group['Fim_de_Semana'].value_counts()

# Extract the counts
num_ocorrencias_fim_de_semana = num_ocorrencias[True] if True in num_ocorrencias else 0
num_ocorrencias_dia_util = num_ocorrencias[False] if False in num_ocorrencias else 0

print(f"Número de ocorrências em fins de semana: {num_ocorrencias_fim_de_semana}")
print(f"Número de ocorrências em dias úteis: {num_ocorrencias_dia_util}")



In [ ]:
#Insert column with Portugal holidays    
portugal_holidays = holidays.Portugal(years=range(2018, 2024))  

# Function to check if the date is a holiday
def is_holiday(date):
    return date in portugal_holidays

# Apply the function to create the 'Feriado' column
gopidataset_group['Feriado'] = gopidataset_group['dt_registo'].apply(is_holiday)
gopidataset_group.head(1)

In [ ]:
# Season function:
def get_season(date):
    year = date.year
    seasons = {'primavera': pd.date_range(start=f'{year}-03-01', end=f'{year}-05-31'),
               'verao': pd.date_range(start=f'{year}-06-01', end=f'{year}-08-31'),
               'outono': pd.date_range(start=f'{year}-09-01', end=f'{year}-11-30')}
    if date in seasons['primavera']:
        return 'Primavera'
    elif date in seasons['verao']:
        return 'Verao'
    elif date in seasons['outono']:
        return 'Outono'
    else:
        return 'Inverno'

# Apply function to create 'Estação' column
gopidataset_group['Estacao'] = gopidataset_group['dt_registo'].apply(get_season)


# Transform the Season column into several "Primavera, Verao, Outono, Inverno"
gopidataset_group['Inverno'] = gopidataset_group['Estacao'] == 'Inverno'
gopidataset_group['Primavera'] = gopidataset_group['Estacao'] == 'Primavera'
gopidataset_group['Verao'] = gopidataset_group['Estacao'] == 'Verão'
gopidataset_group['Outono'] = gopidataset_group['Estacao'] == 'Outono'


gopidataset_group.head(1)
gopidataset_group=gopidataset_group.drop(columns=['Estacao'])

In [ ]:
# Create new data model - With "Freguesia" level indexing granularity
gopidataset_group.to_csv(manipulated_data_directory / 'gopidatasetmodel1_granularidade_freguesia.csv',sep=',', index=False)

In [2]:
###################################################### FP-Growth #########################################################
df = pd.read_csv(manipulated_data_directory / 'gopidatasetmodel1_granularidade_freguesia.csv')
df.head(2)


# Number of lines
num_linhas = len(df)
print(f"Número de linhas: {num_linhas}")


Número de linhas: 774072


In [5]:
df.head(2)

,dt_registo,Subseccao,Freguesia,BNFDRB,CA,CDPC2RD,CDRD,CDEEP,DDP,EOVRDJOPANVP,...,RMPDR,RRPDR,SOOLA,ÁAORM,Fim_de_Semana,Feriado,Inverno,Primavera,Verao,Outono
0,2018-01-01,11066202005,Parque das Nações,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,False,True,True,False,False,False
1,2018-01-01,11061802202,Lumiar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,False,True,True,False,False,False


In [6]:

# Count occurences by season
inverno_count = df['Inverno'].sum()
primavera_count = df['Primavera'].sum()
verao_count = df['Verao'].sum()
outono_count = df['Outono'].sum()

# Identify the season with the most occurrences
estacao_mais_frequente = max(
    {'Inverno': inverno_count, 
     'Primavera': primavera_count, 
     'Verão': verao_count, 
     'Outono': outono_count},
    key=lambda x: x[1]
)

# Filter the data for the most frequent season
df_estacao = df[df[estacao_mais_frequente] == True]

# Count the occurrences of each event in the most frequent season
eventos_comuns = df_estacao.iloc[:, 3:].sum().sort_values(ascending=False)

print(f"Estação com mais ocorrências: {estacao_mais_frequente}")
print("Eventos mais comuns nessa estação:")
print(eventos_comuns)

Estação com mais ocorrências: Outono
Eventos mais comuns nessa estação:
RMPDR            1990155.0
EOVRDJOPANVP      906069.0
Outono            212389.0
CA                161510.0
RNADRDDRSU        157359.0
PED               144911.0
RJPDR             136171.0
LDVPDDPVELDVP     124912.0
RSRPDP            111067.0
G                 106712.0
CDRD               93513.0
SOOLA              85958.0
ÁAORM              83202.0
DDP                82291.0
CDPC2RD            76484.0
EA                 70689.0
FDDDCDPC2ROSM      70276.0
RRPDR              61548.0
OIEVPER            44785.0
BNFDRB             41232.0
CDEEP              37896.0
Fim_de_Semana      21216.0
Feriado             1779.0
Inverno                0.0
Primavera              0.0
Verao                  0.0
dtype: float64


In [3]:
# Change for boolean type
for col in df.select_dtypes(include='number').columns:
           df[col] = df[col].astype(bool)
        
# Define relevant columns for Apriori analysis
relevant_columns =['BNFDRB', 'CA', 'CDPC2RD', 'CDRD', 'CDEEP', 'DDP', 'EOVRDJOPANVP', 'EA',
       'FDDDCDPC2ROSM', 'G', 'LDVPDDPVELDVP', 'OIEVPER', 'PED', 'RNADRDDRSU',
       'RSRPDP', 'RJPDR', 'RMPDR', 'RRPDR', 'SOOLA', 'ÁAORM', 'Feriado', 'Fim_de_Semana', 'Inverno', 'Primavera', 'Verao', 'Outono']

# Associate with basket
basket = df[relevant_columns]

# Apply the FP-Growth algorithm with minimal support
frequent_itemsets = fpgrowth(basket, min_support=0.01, use_colnames=True)
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
print("Itemsets frequentes:\n", frequent_itemsets.head(30))


Itemsets frequentes:
         support                           itemsets
11     0.932356                            (RMPDR)
6      0.694142                     (EOVRDJOPANVP)
2158   0.661606              (EOVRDJOPANVP, RMPDR)
12     0.425990                            (RJPDR)
4711   0.421302                     (RJPDR, RMPDR)
15     0.404539                              (PED)
6134   0.392476                       (PED, RMPDR)
0      0.362783                               (CA)
13     0.358279                       (RNADRDDRSU)
4714   0.352240                (RNADRDDRSU, RMPDR)
26     0.347463                        (CA, RMPDR)
4712   0.324520              (RJPDR, EOVRDJOPANVP)
4713   0.322101       (RJPDR, EOVRDJOPANVP, RMPDR)
22     0.315289                    (LDVPDDPVELDVP)
6135   0.305743                (PED, EOVRDJOPANVP)
10723  0.300809             (LDVPDDPVELDVP, RMPDR)
6137   0.299202         (PED, EOVRDJOPANVP, RMPDR)
8      0.297326                             (CDRD)
2689   0.

In [10]:
# Generate association rules with minimum 1% support
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=0.01)
rules = rules.sort_values(by=['lift', 'confidence'], ascending=[False, False])
print("Regras de associação:\n", rules.head(30))

Regras de associação:
                                      antecedents  \
51581      (FDDDCDPC2ROSM, LDVPDDPVELDVP, RMPDR)   
51596                        (RNADRDDRSU, SOOLA)   
51589             (FDDDCDPC2ROSM, LDVPDDPVELDVP)   
51588                 (RMPDR, RNADRDDRSU, SOOLA)   
51233             (FDDDCDPC2ROSM, LDVPDDPVELDVP)   
51236                        (RNADRDDRSU, SOOLA)   
50511                (FDDDCDPC2ROSM, PED, RMPDR)   
50522                        (RNADRDDRSU, SOOLA)   
50518                       (FDDDCDPC2ROSM, PED)   
50515                 (RMPDR, RNADRDDRSU, SOOLA)   
49920                       (FDDDCDPC2ROSM, PED)   
49925                        (RNADRDDRSU, SOOLA)   
51585         (LDVPDDPVELDVP, RNADRDDRSU, SOOLA)   
51592                     (FDDDCDPC2ROSM, RMPDR)   
51583              (FDDDCDPC2ROSM, RMPDR, SOOLA)   
51594                (LDVPDDPVELDVP, RNADRDDRSU)   
51590                     (FDDDCDPC2ROSM, SOOLA)   
51587         (LDVPDDPVELDVP, RNADRDDRSU,

In [14]:
# This code only serves to present the data in a shorter form in chapter 7 of the dissertation

# Select main columns
rules_selected = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]


# View the formatted table
print("Regras de associação (Top 25):\n")
print(rules_selected.head(25))

Regras de associação (Top 25):

                                     antecedents  \
51581      (FDDDCDPC2ROSM, LDVPDDPVELDVP, RMPDR)   
51596                        (RNADRDDRSU, SOOLA)   
51589             (FDDDCDPC2ROSM, LDVPDDPVELDVP)   
51588                 (RMPDR, RNADRDDRSU, SOOLA)   
51233             (FDDDCDPC2ROSM, LDVPDDPVELDVP)   
51236                        (RNADRDDRSU, SOOLA)   
50511                (FDDDCDPC2ROSM, PED, RMPDR)   
50522                        (RNADRDDRSU, SOOLA)   
50518                       (FDDDCDPC2ROSM, PED)   
50515                 (RMPDR, RNADRDDRSU, SOOLA)   
49920                       (FDDDCDPC2ROSM, PED)   
49925                        (RNADRDDRSU, SOOLA)   
51585         (LDVPDDPVELDVP, RNADRDDRSU, SOOLA)   
51592                     (FDDDCDPC2ROSM, RMPDR)   
51583              (FDDDCDPC2ROSM, RMPDR, SOOLA)   
51594                (LDVPDDPVELDVP, RNADRDDRSU)   
51590                     (FDDDCDPC2ROSM, SOOLA)   
51587         (LDVPDDPVELDVP, RN

In [16]:
# Rules, filter = 'Fim_de_Semana'
rules_weekend = rules[(rules['antecedents'].apply(lambda x: 'Fim_de_Semana' in str(x))) |
                      (rules['consequents'].apply(lambda x: 'Fim_de_Semana' in str(x)))]

print("Regras que envolvem 'Fim_de_Semana':\n", rules_weekend.head(20))

Regras que envolvem 'Fim_de_Semana':
                          antecedents                    consequents  \
45210                (Fim_de_Semana)                      (Inverno)   
45211                      (Inverno)                (Fim_de_Semana)   
28920                (Fim_de_Semana)                       (Outono)   
28921                       (Outono)                (Fim_de_Semana)   
43973                (Fim_de_Semana)                    (Primavera)   
43972                    (Primavera)                (Fim_de_Semana)   
49884                (Fim_de_Semana)                            (G)   
49885                            (G)                (Fim_de_Semana)   
24094                 (EOVRDJOPANVP)         (Fim_de_Semana, RMPDR)   
24091         (Fim_de_Semana, RMPDR)                 (EOVRDJOPANVP)   
39721                (Fim_de_Semana)                (LDVPDDPVELDVP)   
39720                (LDVPDDPVELDVP)                (Fim_de_Semana)   
5973                  (EOVRDJOPANVP)   

In [19]:
# This code only serves to present the data in a shorter form in chapter 7 of the dissertation

# Select main columns
rules_selected = rules_weekend[['antecedents', 'consequents', 'support', 'confidence', 'lift']]


# View the formatted table
print("Regras que envolvem 'Fim_de_Semana:\n")
print(rules_selected.head(25))

Regras que envolvem 'Fim_de_Semana:

                         antecedents                    consequents   support  \
45210                (Fim_de_Semana)                      (Inverno)  0.022635   
45211                      (Inverno)                (Fim_de_Semana)  0.022635   
28920                (Fim_de_Semana)                       (Outono)  0.027408   
28921                       (Outono)                (Fim_de_Semana)  0.027408   
43973                (Fim_de_Semana)                    (Primavera)  0.022954   
43972                    (Primavera)                (Fim_de_Semana)  0.022954   
49884                (Fim_de_Semana)                            (G)  0.021635   
49885                            (G)                (Fim_de_Semana)  0.021635   
24094                 (EOVRDJOPANVP)         (Fim_de_Semana, RMPDR)  0.029539   
24091         (Fim_de_Semana, RMPDR)                 (EOVRDJOPANVP)  0.029539   
39721                (Fim_de_Semana)                (LDVPDDPVELDVP)  0.0

In [20]:
#Filter rules involving 'Seasons'
seasons = ['Inverno', 'Verao', 'Primavera', 'Outono']
rules_seasons = rules[rules['antecedents'].apply(lambda x: any(season in str(x) for season in seasons)) |
                      rules['consequents'].apply(lambda x: any(season in str(x) for season in seasons))]

print("Regras que envolvem estações:\n", rules_seasons.head(30))

Regras que envolvem estações:
                                  antecedents  \
58064                        (Outono, SOOLA)   
58061          (FDDDCDPC2ROSM, EOVRDJOPANVP)   
58059          (Outono, EOVRDJOPANVP, SOOLA)   
58066                        (FDDDCDPC2ROSM)   
43825                 (SOOLA, Outono, RMPDR)   
43832                        (FDDDCDPC2ROSM)   
43830                        (Outono, SOOLA)   
43827                 (FDDDCDPC2ROSM, RMPDR)   
41881                        (FDDDCDPC2ROSM)   
41880                        (Outono, SOOLA)   
56765             (LDVPDDPVELDVP, Primavera)   
56768                                (CDEEP)   
43365     (Outono, PED, EOVRDJOPANVP, RMPDR)   
43388            (LDVPDDPVELDVP, RNADRDDRSU)   
58494          (Outono, EOVRDJOPANVP, RMPDR)   
58475        (LDVPDDPVELDVP, RNADRDDRSU, CA)   
43381            (Outono, PED, EOVRDJOPANVP)   
43372     (LDVPDDPVELDVP, RNADRDDRSU, RMPDR)   
42959            (Outono, PED, EOVRDJOPANVP)   
42962    

In [21]:
# This code only serves to present the data in a shorter form in chapter 7 of the dissertation

# Select main columns
rules_selected = rules_seasons[['antecedents', 'consequents', 'support', 'confidence', 'lift']]


# View the formatted table
print("Regras que envolvem as estações do ano:\n")
print(rules_selected.head(25))

Regras que envolvem as estações do ano:

                              antecedents                         consequents  \
58064                     (Outono, SOOLA)       (FDDDCDPC2ROSM, EOVRDJOPANVP)   
58061       (FDDDCDPC2ROSM, EOVRDJOPANVP)                     (Outono, SOOLA)   
58059       (Outono, EOVRDJOPANVP, SOOLA)                     (FDDDCDPC2ROSM)   
58066                     (FDDDCDPC2ROSM)       (Outono, EOVRDJOPANVP, SOOLA)   
43825              (SOOLA, Outono, RMPDR)                     (FDDDCDPC2ROSM)   
43832                     (FDDDCDPC2ROSM)              (SOOLA, Outono, RMPDR)   
43830                     (Outono, SOOLA)              (FDDDCDPC2ROSM, RMPDR)   
43827              (FDDDCDPC2ROSM, RMPDR)                     (Outono, SOOLA)   
41881                     (FDDDCDPC2ROSM)                     (Outono, SOOLA)   
41880                     (Outono, SOOLA)                     (FDDDCDPC2ROSM)   
56765          (LDVPDDPVELDVP, Primavera)                           

In [14]:
# Rules, filtro = 'Feriado'
rules_feriado = rules[(rules['antecedents'].apply(lambda x: 'Feriado' in str(x))) |
                      (rules['consequents'].apply(lambda x: 'Feriado' in str(x)))]

print("Regras que envolvem 'Feriado':\n", rules_feriado)

Regras que envolvem 'Feriado':
 Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction, zhangs_metric]
Index: []
